In [1]:
import os
import sys
#nb_dir = os.path.split(os.getcwd())[0]
#if nb_dir not in sys.path:
#    sys.path.append(nb_dir)
    
autolib_dir = '/home/kai/data/shiyi/AlphaBoosting/automl'
if autolib_dir not in sys.path:
    sys.path.append(autolib_dir)


In [2]:
sys.path

['',
 '/home/kai/anaconda3/envs/tf_gpu/lib/python35.zip',
 '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5',
 '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/plat-linux',
 '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/lib-dynload',
 '/home/kai/.local/lib/python3.5/site-packages',
 '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages',
 '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/cycler-0.10.0-py3.5.egg',
 '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/IPython/extensions',
 '/home/kai/.ipython',
 '/home/kai/data/shiyi/AlphaBoosting/automl']

In [3]:
%load_ext autoreload

In [4]:
%autoreload 2

In [5]:
from automl_libs import feature_engineering as fe
from automl_libs import encoding, kernels, nn_libs 

In [6]:
categorical_features = ['Month', 'DayofMonth', 'DayOfWeek']#, 'DepTime', 'UniqueCarrier', 'Origin', 'Dest', 'Distance']

In [7]:
import itertools
#params = {'split_col': 't', 'coefficient': 10, 'n': 2, 'fillna': 22}
def get_features_to_gen(function_list):
    features_to_gen = []
    for function in function_list:
        for i in range(2, len(categorical_features)+1):
            for combine in itertools.combinations(categorical_features, i):
                if function.__name__ == 'count_std_over_mean':
                    features_to_gen.append({'params': {'coefficient': 10}, 'function': function, 'feature_cols': list(combine)})
                else:
                    features_to_gen.append({'params': {}, 'function': function, 'feature_cols': list(combine)})
                
    return features_to_gen

#features_to_gen = get_features_to_gen([fe.count, fe.unique_count, fe.cumulative_count,
#                                      fe.reverse_cumulative_count, fe.variance, fe.count_std_over_mean])
features_to_gen = get_features_to_gen([fe.count])

features_to_gen

[{'feature_cols': ['Month', 'DayofMonth'],
  'function': <function automl_libs.feature_engineering.count>,
  'params': {}},
 {'feature_cols': ['Month', 'DayOfWeek'],
  'function': <function automl_libs.feature_engineering.count>,
  'params': {}},
 {'feature_cols': ['DayofMonth', 'DayOfWeek'],
  'function': <function automl_libs.feature_engineering.count>,
  'params': {}},
 {'feature_cols': ['Month', 'DayofMonth', 'DayOfWeek'],
  'function': <function automl_libs.feature_engineering.count>,
  'params': {}}]

In [8]:
len(features_to_gen)

4

In [9]:
project_path = '/home/kai/data/shiyi/AlphaBoosting/automl/automl_app/project1/' 

In [10]:
from automl_app import logger_config

logger_config.config(project_path+'log_project1.log')

In [11]:
def lgb_gs_params_gen():
    import time
    import numpy as np
    seed = int(time.time()* 1000000) % 45234634
    np.random.seed(seed)
    params = {    
        'objective': 'binary',
        'boosting': 'gbdt',
        'num_rounds': 20,
        'learning_rate': np.random.choice([0.1,0.03]),#0.001]),
        'num_leaves': np.random.choice([15,31,61,127]),
        'num_threads': 8, # best speed: set to number of real cpu cores, which is vCPU/2
        'max_depth': -1, # no limit. This is used to deal with over-fitting when #data is small.
        'min_data_in_leaf': np.random.randint(20,50),  #minimal number of data in one leaf. 
        'feature_fraction': np.random.randint(4,11)/10,
        'feature_fraction_seed': seed,
        'early_stopping_round':70,
        'bagging_freq': 1, #0 means disable bagging. k: perform bagging at every k iteration
        'bagging_fraction': np.random.randint(4,11)/10, #Randomly select part of data 
        'bagging_seed': seed,
        'scale_pos_weight': 2,
        'metric' : 'auc' 
    }
    return params, seed

In [12]:
lgb_gs_params_gen()

({'bagging_fraction': 0.5,
  'bagging_freq': 1,
  'bagging_seed': 33118970,
  'boosting': 'gbdt',
  'early_stopping_round': 70,
  'feature_fraction': 0.4,
  'feature_fraction_seed': 33118970,
  'learning_rate': 0.1,
  'max_depth': -1,
  'metric': 'auc',
  'min_data_in_leaf': 35,
  'num_leaves': 31,
  'num_rounds': 20,
  'num_threads': 8,
  'objective': 'binary',
  'scale_pos_weight': 2},
 33118970)

In [86]:
from automl_app.app import AlphaBoosting

automl_config_file = project_path + 'automl_config.json'
run_record_file_name = project_path + 'last_run_record.json' # don't created this file
AlphaBoosting(automl_config_file, features_to_gen, lgb_gs_params_gen)

2018-06-21 20:21:28 EST | INFO     | automl_app.app.AlphaBoosting                 | __init__             | #58  | Run record file [./output/last_run_record.json] found. Continue from the previous run...
2018-06-21 20:21:28 EST | INFO     | automl_app.app.AlphaBoosting                 | __init__             | #88  | save run record
2018-06-21 20:21:28 EST | INFO     | automl_app.app.AlphaBoosting                 | _save_run_record     | #153 | val index is saved at ./temp_data/val_index.pkl
2018-06-21 20:21:28 EST | INFO     | automl_app.app.AlphaBoosting                 | _save_run_record     | #155 | run record is saved at ./output/last_run_record.json
2018-06-21 20:21:29 EST | INFO     | automl_app.app.AlphaBoosting                 | __init__             | #92  | generate todo list
2018-06-21 20:21:29 EST | INFO     | automl_app.app.AlphaBoosting                 | __init__             | #108 | feature engineering
2018-06-21 20:21:29 EST | INFO     | automl_app.app.AlphaBoosting      

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Dest (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
Origin (InputLayer)             (None, 1)            0                                            
__________________________________________________________________________________________________
UniqueCarrier (InputLayer)      (None, 1)            0                                            
__________________________________________________________________________________________________
DepTime (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
Month (Inp

2018-06-21 20:21:40 EST | DEBUG    | automl_libs.nn_libs.LearningRateTracker      | _show_lr             | #138 | After this epoch: init lr 0.010000, lr_decay 0.259921, iterations 5, current lr 0.004349



Epoch 00001: roc_auc_val improved from -inf to 0.61849, saving model to ./output/nn_models/nn_1529626890.hdf5
 - 13s - loss: 0.6946 - acc: 0.5314 - val_loss: 0.6464 - val_acc: 0.7153 - roc_auc_val: 0.6185
